In [64]:
# Add project root to sys.path so we can import from src/
import sys
from pathlib import Path
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
sys.modules.pop("src.data_processing", None)
sys.modules.pop("src.models", None)

from src.data_processing import load_data, detect_missing_mask
from src.models import manual_ordinal_encode, get_outlier_mask_iqr, independent_ttest_numpy, chi_square_test_numpy
import numpy as np

In [65]:
exp_map = {'<1': 0}
for i in range(1, 21): exp_map[str(i)] = i
exp_map['>20'] = 21

lnj_map = {'never': 0}
for i in range(1, 5): lnj_map[str(i)] = i
lnj_map['>4'] = 5

company_size_map = {
    '<10': 0, 
    '10-49': 1, 
    '50-99': 2, 
    '100-500': 3, 
    '500-999': 4, 
    '1000-4999': 5, 
    '5000-9999': 6, 
    '10000+': 7
}

header, data = load_data('../data/raw/aug_train.csv')

# 1. Lấy biến target (y)
y_raw = data[:, header.index('target')].astype(float).copy()

# 2. Loại bỏ cột 'enrollee_id' và 'target' khỏi X
id_idx = header.index('enrollee_id')
target_idx = header.index('target')
X_raw = np.delete(data, [id_idx, target_idx], axis=1)
header_X = np.delete(header, [id_idx, target_idx])

# 3. Khai báo nhóm cột
numerical_cols = ['city_development_index', 'training_hours']
ordinal_cols = ['experience', 'last_new_job', 'company_size']
categorical_cols = [col for col in header_X if col not in numerical_cols + ordinal_cols]

print(f"✅ Tải dữ liệu thành công. \nX_raw shape: {X_raw.shape}\ny_raw shape: {y_raw.shape}")

✅ Tải dữ liệu thành công. 
X_raw shape: (19158, 12)
y_raw shape: (19158,)


In [66]:
print("\n--- KIỂM ĐỊNH CHI-SQUARE CHO GENDER (Feature Selection) ---\n")

# Lấy chỉ mục của cột 'gender' (Giả sử nó nằm trong categorical_cols)
gender_idx = header_X.tolist().index('gender')

# Lấy cột gender thô (trước khi OHE) và target
gender_col_raw = X_raw[:, gender_idx].copy()
target_col = y_raw.copy()

chi_stat, df, observed_table = chi_square_test_numpy(gender_col_raw, target_col)

# Ngưỡng tới hạn (Critical Value) cho alpha=0.05 và df=3
critical_value = 7.82

print(f"Chi-square Statistic: {chi_stat:.4f}")
print(f"Degrees of Freedom (df): {df}")
print(f"Ngưỡng Tới hạn (alpha=0.05): {critical_value:.2f}")

if chi_stat > critical_value:
    print("✅ Quyết định: Bác bỏ H0. Giới tính có mối quan hệ phụ thuộc với quyết định đổi việc.")
    # (Hành động: Giữ lại cột gender)
else:
    print("❌ Quyết định: Chưa đủ bằng chứng bác bỏ H0 (|Chi-sq| < 5.99).")
    print("   -> Kết luận sơ bộ: Giới tính có thể độc lập với quyết định đổi việc.")
    # (Hành động: Cân nhắc loại bỏ cột gender để giảm nhiễu)

print("\nBảng Tần suất Quan sát (Observed Table):\n", observed_table)


--- KIỂM ĐỊNH CHI-SQUARE CHO GENDER (Feature Selection) ---

Chi-square Statistic: 117.3772
Degrees of Freedom (df): 3
Ngưỡng Tới hạn (alpha=0.05): 7.82
✅ Quyết định: Bác bỏ H0. Giới tính có mối quan hệ phụ thuộc với quyết định đổi việc.

Bảng Tần suất Quan sát (Observed Table):
 [[ 3119.  1389.]
 [  912.   326.]
 [10209.  3012.]
 [  141.    50.]]


In [67]:
# --- KHỞI TẠO MA TRẬN TẠM THỜI ---
X_processed_num_ord = [] 
X_processed_cat = []     
y_aligned = y_raw.copy()

print("\n--- BƯỚC 1: Xử lý Missing Values, Lỗi Dữ liệu & Mã hóa ---")

# --- 1.1 XỬ LÝ CỘT SỐ VÀ CỘT THỨ TỰ (Median Imputation) ---
for colname in numerical_cols + ordinal_cols:
    col_idx = header_X.tolist().index(colname)
    col = X_raw[:, col_idx].copy()
    
    # 1. Mã hóa Ordinal & Kiểm tra tính hợp lệ (trả về NaN nếu lỗi)
    if colname in ordinal_cols:
            if colname == 'experience':
                map_dict = exp_map
            elif colname == 'last_new_job':
                map_dict = lnj_map
            elif colname == 'company_size': 
                map_dict = company_size_map 
                
            col_numeric = manual_ordinal_encode(col, map_dict)
    else:
        missing_mask_initial = detect_missing_mask(col)
        col_numeric = col.astype(str)
        col_numeric[missing_mask_initial] = np.nan
        col_numeric = col_numeric.astype(float)
        
    # 2. Xử lý Missing Values (Median Imputation)
    missing_mask = np.isnan(col_numeric) 
    median_val = np.nanmedian(col_numeric) 
    col_numeric[missing_mask] = median_val
    
    X_processed_num_ord.append(col_numeric.reshape(-1, 1))
    print(f"Đã xử lý thiếu & mã hóa cho {colname}. Median: {median_val:.2f}")


# --- 1.2 XỬ LÝ CỘT PHÂN LOẠI (Specific Value Imputation & Lỗi) ---
for colname in categorical_cols:
    col_idx = header_X.tolist().index(colname)
    col = X_raw[:, col_idx].copy()
        
    # Xử lý Missing Values: Filling bằng 'Unknown'
    missing_mask = detect_missing_mask(col)
    col[missing_mask] = 'Unknown'
    
    X_processed_cat.append(col.reshape(-1, 1))
    print(f"Đã xử lý thiếu cho {colname}. Điền bằng 'Unknown'.")

X_temp_num_ord = np.hstack(X_processed_num_ord).astype(float)
X_temp_cat = np.hstack(X_processed_cat)


--- BƯỚC 1: Xử lý Missing Values, Lỗi Dữ liệu & Mã hóa ---
Đã xử lý thiếu & mã hóa cho city_development_index. Median: 0.90
Đã xử lý thiếu & mã hóa cho training_hours. Median: 47.00
Đã xử lý thiếu & mã hóa cho experience. Median: 9.00
Đã xử lý thiếu & mã hóa cho last_new_job. Median: 1.00
Đã xử lý thiếu & mã hóa cho company_size. Median: 3.00
Đã xử lý thiếu cho city. Điền bằng 'Unknown'.
Đã xử lý thiếu cho gender. Điền bằng 'Unknown'.
Đã xử lý thiếu cho relevent_experience. Điền bằng 'Unknown'.
Đã xử lý thiếu cho enrolled_university. Điền bằng 'Unknown'.
Đã xử lý thiếu cho education_level. Điền bằng 'Unknown'.
Đã xử lý thiếu cho major_discipline. Điền bằng 'Unknown'.
Đã xử lý thiếu cho company_type. Điền bằng 'Unknown'.


In [68]:
print("\n--- BƯỚC 2: Xử lý Outliers (IQR) ---")

# Mask tổng hợp cho toàn bộ tập dữ liệu
overall_valid_mask = np.ones(X_temp_num_ord.shape[0], dtype=bool)

# Áp dụng kiểm tra Outlier cho từng cột số/thứ tự
for i, colname in enumerate(numerical_cols): # CHỈ DUYỆT TRÊN numerical_cols
    col = X_temp_num_ord[:, i]
    valid_mask_col = get_outlier_mask_iqr(col)
    
    # Cập nhật mask tổng hợp
    overall_valid_mask = overall_valid_mask & valid_mask_col
    
    print(f"Outliers detected in {colname}: {np.sum(~valid_mask_col)} rows.")

# Loại bỏ các hàng chứa Outlier và đồng bộ X và y
if np.sum(~overall_valid_mask) > 0:
    X_temp_num_ord_filtered = X_temp_num_ord[overall_valid_mask]
    X_temp_cat_filtered = X_temp_cat[overall_valid_mask]
    y_aligned_filtered = y_aligned[overall_valid_mask]
    print(f"Đã loại bỏ {np.sum(~overall_valid_mask)} hàng chứa Outlier.")
else:
    X_temp_num_ord_filtered = X_temp_num_ord
    X_temp_cat_filtered = X_temp_cat
    y_aligned_filtered = y_aligned
    print("Không cần loại bỏ Outlier. Tiếp tục với toàn bộ dữ liệu.")


--- BƯỚC 2: Xử lý Outliers (IQR) ---
Outliers detected in city_development_index: 17 rows.
Outliers detected in training_hours: 984 rows.
Đã loại bỏ 1001 hàng chứa Outlier.


In [69]:
print("\n--- BƯỚC 3: Standardization (Z-score) ---")

X_std = X_temp_num_ord_filtered.copy()
num_ord_indices = np.arange(X_std.shape[1])

for i in num_ord_indices:
    col = X_std[:, i]
    
    mean_val = np.mean(col)
    std_val = np.std(col)
    
    # Chuẩn hóa Z-score: (X - mean) / std (CHỈ DÙNG NUMPY)
    X_std[:, i] = (col - mean_val) / (std_val + 1e-6) # Ổn định số học
    
print("✅ Standardization hoàn thành cho các cột số và thứ tự.")


--- BƯỚC 3: Standardization (Z-score) ---
✅ Standardization hoàn thành cho các cột số và thứ tự.


In [70]:
print("\n--- BƯỚC 4A: Feature Engineering ---")

# Lấy các cột đã chuẩn hóa (City Dev Index và Experience)
city_dev_std = X_std[:, 0]
training_hours_std = X_std[:, 1]
experience_std = X_std[:, 2]

capped_experience_std = np.maximum(experience_std, -0.9999)
opportunity_gap = np.log1p(capped_experience_std) / (city_dev_std + 1e-6) 
training_experience_ratio = training_hours_std / (experience_std + 1e-6)

X_std_fe = np.hstack((
    X_std, 
    opportunity_gap.reshape(-1, 1), 
    training_experience_ratio.reshape(-1, 1) 
))
print(f"Đã thêm cột Opportunity_Gap và Training_Experience_Ratio. Kích thước X_std mới: {X_std_fe.shape}")


print("-> Chuẩn hóa lại các cột Feature Engineering...")

# Chỉ mục của 2 cột FE mới: -2 (Opportunity Gap) và -1 (Training Ratio)
fe_cols_to_std = X_std_fe[:, -2:].copy() # Lấy 2 cột cuối cùng

# Chuẩn hóa lại từng cột
for i in range(fe_cols_to_std.shape[1]):
    col = fe_cols_to_std[:, i]
    mean_val = np.mean(col)
    std_val = np.std(col)
    
    # Áp dụng Z-score (CHỈ DÙNG NUMPY)
    fe_cols_to_std[:, i] = (col - mean_val) / (std_val + 1e-6)


# Thay thế các cột FE chưa chuẩn hóa trong X_std_fe bằng các cột đã chuẩn hóa
X_std_fe_final = np.hstack((
    X_std_fe[:, :-2],        # Các cột gốc đã chuẩn hóa
    fe_cols_to_std           # Các cột FE đã chuẩn hóa lại
))

print(f"✅ Đã chuẩn hóa lại cột FE. Kích thước X_std_fe_final: {X_std_fe_final.shape}")


# --- BƯỚC 4B: One-Hot Encoding (OHE) ---
print("\n--- BƯỚC 4B: One-Hot Encoding ---")

X_final_list = [X_std_fe_final] # Bắt đầu với các cột đã chuẩn hóa và FE

# Áp dụng OHE cho các cột phân loại
for i in range(X_temp_cat_filtered.shape[1]):
    col = X_temp_cat_filtered[:, i]
    unique_values = np.unique(col)
    
    for val in unique_values:
        # Tạo cột nhị phân (0 hoặc 1)
        one_hot_col = (col == val).astype(float)
        X_final_list.append(one_hot_col.reshape(-1, 1))
        
        
# 5. Kết hợp X cuối cùng
X = np.hstack(X_final_list)
y = y_aligned_filtered.reshape(-1, 1) # y cuối cùng
print(f"✅ Preprocessing hoàn thành. X cuối cùng shape: {X.shape}, y cuối cùng shape: {y.shape}")


--- BƯỚC 4A: Feature Engineering ---
Đã thêm cột Opportunity_Gap và Training_Experience_Ratio. Kích thước X_std mới: (18157, 7)
-> Chuẩn hóa lại các cột Feature Engineering...
✅ Đã chuẩn hóa lại cột FE. Kích thước X_std_fe_final: (18157, 7)

--- BƯỚC 4B: One-Hot Encoding ---
✅ Preprocessing hoàn thành. X cuối cùng shape: (18157, 158), y cuối cùng shape: (18157, 1)


In [71]:
print("--- Thống kê Mô tả (Sau Standardization) ---")

# (Phần thống kê mô tả giữ nguyên)
city_dev_std = X[:, 0] 
print(f"Kiểm tra cột City Development Index:")
print(f"  Min: {np.min(city_dev_std):.4f}, Max: {np.max(city_dev_std):.4f}")
print(f"  Mean: {np.mean(city_dev_std):.4f}, Std Dev: {np.std(city_dev_std):.4f}\n") 

training_dev_std = X[:, 1]
print(f"Kiểm tra cột Training Hours:")
print(f"  Min: {np.min(training_dev_std):.4f}, Max: {np.max(training_dev_std):.4f}")
print(f"  Mean: {np.mean(training_dev_std):.4f}, Std Dev: {np.std(training_dev_std):.4f}\n")

exp_dev_std = X[:, 2]
print(f"Kiểm tra cột Experience:")
print(f"  Min: {np.min(exp_dev_std):.4f}, Max: {np.max(exp_dev_std):.4f}")
print(f"  Mean: {np.mean(exp_dev_std):.4f}, Std Dev: {np.std(exp_dev_std):.4f}\n")

lnj_dev_std = X[:, 3]
print(f"Kiểm tra cột Last New Job:")
print(f"  Min: {np.min(lnj_dev_std):.4f}, Max: {np.max(lnj_dev_std):.4f}")
print(f"  Mean: {np.mean(lnj_dev_std):.4f}, Std Dev: {np.std(lnj_dev_std):.4f}\n")

comp_size_dev_std = X[:, 4]
print(f"Kiểm tra cột Company Size:")
print(f"  Min: {np.min(comp_size_dev_std):.4f}, Max: {np.max(comp_size_dev_std):.4f}")
print(f"  Mean: {np.mean(comp_size_dev_std):.4f}, Std Dev: {np.std(comp_size_dev_std):.4f}\n")

opportunity_gap = X[:, 5]
print(f"Kiểm tra cột Opportunity Gap:")
print(f"  Min: {np.min(opportunity_gap):.4f}, Max: {np.max(opportunity_gap):.4f}")
print(f"  Mean: {np.mean(opportunity_gap):.4f}, Std Dev: {np.std(opportunity_gap):.4f}\n")

training_experience_ratio = X[:, 6]
print(f"Kiểm tra cột Training-Experience Ratio:")
print(f"  Min: {np.min(training_experience_ratio):.4f}, Max: {np.max(training_experience_ratio):.4f}")
print(f"  Mean: {np.mean(training_experience_ratio):.4f}, Std Dev: {np.std(training_experience_ratio):.4f}")

--- Thống kê Mô tả (Sau Standardization) ---
Kiểm tra cột City Development Index:
  Min: -2.8525, Max: 0.9745
  Mean: 0.0000, Std Dev: 1.0000

Kiểm tra cột Training Hours:
  Min: -1.2883, Max: 3.0369
  Mean: 0.0000, Std Dev: 1.0000

Kiểm tra cột Experience:
  Min: -1.4923, Max: 1.6078
  Mean: 0.0000, Std Dev: 1.0000

Kiểm tra cột Last New Job:
  Min: -1.1907, Max: 1.8111
  Mean: -0.0000, Std Dev: 1.0000

Kiểm tra cột Company Size:
  Min: -1.9419, Max: 2.1374
  Mean: -0.0000, Std Dev: 1.0000

Kiểm tra cột Opportunity Gap:
  Min: -66.4112, Max: 19.5639
  Mean: -0.0000, Std Dev: 1.0000

Kiểm tra cột Training-Experience Ratio:
  Min: -13.6076, Max: 5.6552
  Mean: 0.0000, Std Dev: 1.0000


In [72]:
# 1. Tách dữ liệu Training Hours dựa trên Target (sử dụng y cuối cùng)
y_flat = y.flatten()
hours_target_1 = training_dev_std[y_flat == 1]
hours_target_0 = training_dev_std[y_flat == 0]

print("\n--- KIỂM ĐỊNH GIẢ THIẾT T-TEST (TRAINING HOURS) ---")

# Giả thiết Ho (Null Hypothesis):
print("H0: Trung bình Giờ Đào tạo là bằng nhau giữa nhóm Đổi việc và nhóm Không đổi việc.")
# Giả thiết Ha (Alternative Hypothesis):
print("H1: Trung bình Giờ Đào tạo là khác nhau giữa hai nhóm.")

t_statistic, degrees_of_freedom = independent_ttest_numpy(hours_target_1, hours_target_0)

print(f"\nThông số Kiểm định T-test:")
print(f"  Trung bình TGT=1 (Đổi việc): {np.mean(hours_target_1):.4f}")
print(f"  Trung bình TGT=0 (Không đổi việc): {np.mean(hours_target_0):.4f}")
print(f"  T-Statistic (T): {t_statistic:.4f}")
print(f"  Bậc tự do (df): {degrees_of_freedom:.2f}")

# Nhận định sơ bộ (Quyết định):
# Dựa trên T-value và df, ta có thể tham khảo bảng phân phối T. 
# Nếu T > 1.96 (ngưỡng thô cho df lớn, alpha=0.05), ta có thể bác bỏ H0.

# Ta sử dụng T-statistic tuyệt đối:
if np.abs(t_statistic) > 1.96: 
	print("\n✅ Quyết định Sơ bộ: |T-stat| > 1.96. Có dấu hiệu bác bỏ H0 tại mức ý nghĩa 0.05.")
	print("   -> Kết luận sơ bộ: Có sự khác biệt có ý nghĩa thống kê về số giờ đào tạo giữa hai nhóm.")
else:
	print("\n❌ Quyết định Sơ bộ: |T-stat| <= 1.96. Chưa đủ bằng chứng để bác bỏ H0.")

opportunity_gap_target_1 = opportunity_gap[y_flat == 1]
opportunity_gap_target_0 = opportunity_gap[y_flat == 0]

print("\n--- KIỂM ĐỊNH GIẢ THIẾT T-TEST (OPPORTUNITY GAP) ---")

# Giả thiết Ho (Null Hypothesis):
print("H0: Không có sự khác biệt có ý nghĩa thống kê về Opportunity Gap trung bình giữa nhóm Đổi việc và nhóm Không đổi việc.")
# Giả thiết Ha (Alternative Hypothesis):
print("H1: Có sự khác biệt có ý nghĩa thống kê về Opportunity Gap trung bình giữa hai nhóm.")

t_statistic_og, degrees_of_freedom_og = independent_ttest_numpy(opportunity_gap_target_1, opportunity_gap_target_0)

print(f"\nThông số Kiểm định T-test:")
print(f"  Trung bình TGT=1 (Đổi việc): {np.mean(opportunity_gap_target_1):.4f}")
print(f"  Trung bình TGT=0 (Không đổi việc): {np.mean(opportunity_gap_target_0):.4f}")
print(f"  T-Statistic (T): {t_statistic_og:.4f}")
print(f"  Bậc tự do (df): {degrees_of_freedom_og:.2f}")

# Nhận định sơ bộ (Quyết định):
if np.abs(t_statistic_og) > 1.96: 
	print("\n✅ Quyết định Sơ bộ: |T-stat| > 1.96. Có dấu hiệu bác bỏ H0 tại mức ý nghĩa 0.05.")
	print("   -> Kết luận sơ bộ: Opportunity Gap có sự khác biệt có ý nghĩa thống kê giữa hai nhóm.")
else:
	print("\n❌ Quyết định Sơ bộ: |T-stat| <= 1.96. Chưa đủ bằng chứng để bác bỏ H0.")


--- KIỂM ĐỊNH GIẢ THIẾT T-TEST (TRAINING HOURS) ---
H0: Trung bình Giờ Đào tạo là bằng nhau giữa nhóm Đổi việc và nhóm Không đổi việc.
H1: Trung bình Giờ Đào tạo là khác nhau giữa hai nhóm.

Thông số Kiểm định T-test:
  Trung bình TGT=1 (Đổi việc): -0.0179
  Trung bình TGT=0 (Không đổi việc): 0.0060
  T-Statistic (T): -1.4138
  Bậc tự do (df): 7963.35

❌ Quyết định Sơ bộ: |T-stat| <= 1.96. Chưa đủ bằng chứng để bác bỏ H0.

--- KIỂM ĐỊNH GIẢ THIẾT T-TEST (OPPORTUNITY GAP) ---
H0: Không có sự khác biệt có ý nghĩa thống kê về Opportunity Gap trung bình giữa nhóm Đổi việc và nhóm Không đổi việc.
H1: Có sự khác biệt có ý nghĩa thống kê về Opportunity Gap trung bình giữa hai nhóm.

Thông số Kiểm định T-test:
  Trung bình TGT=1 (Đổi việc): 0.0330
  Trung bình TGT=0 (Không đổi việc): -0.0111
  T-Statistic (T): 3.6382
  Bậc tự do (df): 16830.34

✅ Quyết định Sơ bộ: |T-stat| > 1.96. Có dấu hiệu bác bỏ H0 tại mức ý nghĩa 0.05.
   -> Kết luận sơ bộ: Opportunity Gap có sự khác biệt có ý nghĩa thốn

In [73]:
output_path = '../data/processed/data_processed.npz'
np.savez_compressed(output_path, X_features=X, y_target=y)

print(f"\n✅ Dữ liệu đã xử lý được lưu thành công dưới dạng nhị phân tại {output_path}.")


✅ Dữ liệu đã xử lý được lưu thành công dưới dạng nhị phân tại ../data/processed/data_processed.npz.
